In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset100k, Dataset1m
from src.models import GMFBCEModel, MLPBCEModel, NeuralMatrixFactorizationBCEModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = 'ml-100k'
    # data_dir = 'ml-1m'
    epochs = 40
    batch_size = 2048
    dim = 40
    lr = 0.01
    b1 = 0.9
    b2 = 0.999
    weight_decay = 0.0001

dataset = Dataset100k(config.data_dir)
# dataset = Dataset1m(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    "HR@1": (hitratio, {"top_n": 1}),
    "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    "NDCG@1": (ndcg, {"top_n": 1}),
    "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [3]:
model = NeuralMatrixFactorizationBCEModel(dataset.user_count, dataset.item_count, gmf_embed_size=config.dim, mlp_embed_size=config.dim, layers=[20, 10, 5], alpha=0.5)
gmf_model = GMFBCEModel(dataset.user_count, dataset.item_count, embed_size=config.dim)
mlp_model = MLPBCEModel(dataset.user_count, dataset.item_count, embed_size=config.dim, layers=[20, 10, 5])

model.load_pretrained_weights(gmf_model, mlp_model)

# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(config.b1, config.b2), weight_decay=config.weight_decay)

# trainer = Trainer(dataset, model, optimizer, metrics, 
#                   epochs=config.epochs, batch_size=config.batch_size,
#                   device=device)

In [ ]:
trainer.train(evaluate=True, verbose=True, progressbar=True)
# trainer.test(verbose=False, pbar=False)

In [ ]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

In [ ]:
torch.save(trainer.model.state_dict(), "saved_models/nmfbce.pt")
# trainer.model.load_state_dict(torch.load("saved_models/nmfbce.pt"))